## Imports

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools import Tool

from wikipedia_tools import save_results_to_path, read_results_from_path, get_top_k_keywords
from wikipedia_tools import __all__ as tools_available

## Environment Initialization

In [ ]:
file_save_format: str = """Question: {user question}\nTopic: {topic}\nSummary: {summary of results}""" # format we expect agent to follow when saving results
system_prompt = f"""You are an expert researcher that leverages wikipedia to answer questions. You can use the following tools: {tools_available}. You are not to answer questions without using a tool or using information resulting from a tool. For example, if you are asked "What is the capital of France?", you should use the "search_wikipedia" tool to find the answer. If a user later asks, "What is the capital of France?", you may simply answer based off of your previous findings. In summary, query for new questions, for redundant questions, use your previous findings.

Output guidelines:
You must always respond in the following JSON format:
{{“intent”:…, “status”: ..., “function”: {{ “name”:…, “parameters”:...}}}}
- The "intent" field should be a short description of what you are trying to do.
- The "status" field should be one of the following: "in_progress", "done", or "request_human_approval".
  - "in_progress" means you are still working on the task and will need to call another tool.
  - "done" means you have completed the task and have all the information you need to answer the user's question.
  - "request_human_approval" means you need human approval before proceeding.
- The "function" field should contain the name of the tool being called and its parameters.
  - The parameters should be a dictionary containing the necessary arguments for the tool. For example, "parameters": {{"query": "What is the capital of France?"}}.

File saving format:
For any requests that require saving the results the format should be as followed:
{file_save_format}.
- If the user does not specify a specific file name, you must name the file with the convention wiki_findings_i.txt"""

questions: list[str] = [
  "What is the status of the Sudanese Civil War in August 2025?",
  "Can you save your findings to ./sudan_civil_war.txt?",
  "Tell me the exact contents of the file at ./random_txt.txt.",
  "What are the top 3 words used in the previous text?",
]

In [ ]:
basic_model = OllamaLLM(model="llama3.1") 
basic_model2 = OllamaLLM(model="llama3.1")
basic_model = OllamaLLM(model="llama3.1") 

tools = [
  Tool(name="search_wikipedia", func=wiki_tool.run, description="Searches wikipedia for the topic you provide"), # Searches wikipedia
  save_results_to_path, # save agents results to user specified path
  read_results_from_path, # read agents results from user specified path
  get_top_k_keywords 
]


In [13]:
agent = initialize_agent(
  llm=basic_model, 
  tools=tools, 
  prompt=system_prompt,
  agent="zero-shot-react-description",  # ReAct-style
  verbose=True,
  ) # uses ReAct concept

/tmp/ipykernel_295873/284988254.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


ValueError: ZeroShotAgent does not support multi-input tool save_results_to_path.

### Planning agent example

In [ ]:
query = "Who are the parties involved in the sudanese civil war? What is the historical context of the conflict? Provide a 200 word summary and save the results to ./sudan_historical_analysis.txt."
response = agent.invoke({"messages": [{"role": "user", "content": query}]})
print(f"Question: {query}\nResponse: {response}\n")

### Agent with long-term memory example

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS(embedding_model)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,       # number of characters per chunk
    chunk_overlap=25,      # overlap between chunks to maintain context
)

#### Creating vector store that can retrieve relavent chunks to a query

In [ ]:
sudan_text = read_results_from_path.func("./sudan_civil_war.txt")
chunks = text_splitter.split_text(sudan_text)
docs = [Document(page_content=chunk) for chunk in chunks]
for doc in docs:
    vector_store.add_document(doc)

#### RAG assisted response

In [ ]:
query = "What are the main causes of the Sudanese Civil War?"
relevant_docs = vector_store.similarity_search(query, k=2)
query_with_docs = f"""{query}. The following are documents from previous questions: {relevant_docs}"""

In [ ]:
response = basic_model2.invoke({"input": query_with_docs})
print(f"Question: {query}\nResponse: {response}\n")